In [1]:
import pandas as pd

In [2]:
model_seq_info = pd.read_parquet('../data3/interim/model_seq_info.pq')
search_df = pd.read_parquet('../data3/interim/model_seq_blast_search.pq')

**Query:** train gene

**Target:** test gene

In [3]:
pos_ids = model_seq_info.loc[model_seq_info['defensive'], 'seq_id']

In [4]:
top_train_list = list()
for fold, fold_test_df in model_seq_info.groupby('test_fold'):
    print(fold)
    top_blast_df = (search_df[~search_df['query'].isin(fold_test_df['seq_id']) & 
                              search_df['query'].isin(pos_ids) &
                              search_df['target'].isin(fold_test_df['seq_id'])]
                     .sort_values('bits', ascending=False)
                     .groupby('target')
                     .head(1))
    top_train_list.append(top_blast_df)

0
1
2
3
4


In [5]:
cat_top_hit = pd.concat(top_train_list)

In [6]:
out_df = (model_seq_info[['seq_id']]
          .merge(cat_top_hit.rename(columns={'target': 'seq_id', 
                                                    'bits': 'prediction'})
                 [['seq_id', 'prediction']], 
                 how='left', on='seq_id'))
out_df['prediction'] = out_df['prediction'].fillna(0)
out_df['method'] = 'Defense BLAST search'

In [7]:
out_df.to_parquet('../data3/interim/cv_predictions_defense_blast.pq', index=False)